# Manifold GP Semi-Supervised Learning via Precision Matrix on 2D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from mayavi import mlab
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

Select the device: GPU or CPU.

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset and Generate Ground Truth

In [3]:
data_path = files('manifold_gp.data').joinpath('dragon10k.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)
sampled_x = torch.from_numpy(nodes).float().to(device)
sampled_y = torch.from_numpy(truth).float().to(device)
(m, n) = sampled_x.shape

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold
Setting the variable `manifold_noise` you can apply Gaussian noise to the sampled points from the manifold.

In [4]:
manifold_noise = 0.001
noisy_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth
Setting the variable `function_noise` you can apply Gaussian noise to the training points.

In [5]:
function_noise = 0.005
noisy_y = sampled_y + function_noise * torch.randn(m).to(device)

## Training Dataset

Create the training dataset. You can set the variable `num_train` to define the number of points within your dataset.

In [6]:
num_train = 100
train_idx = torch.randperm(m)[:num_train]
train_x = noisy_x[train_idx, :]
train_y = noisy_y[train_idx]

## Test Dataset

Create the training dataset. You can set the variable `num_test` to define the number of points within your dataset.

In [7]:
num_test = 100
test_idx = torch.randperm(m)[:num_test]
test_x = noisy_x[test_idx, :]
test_y = noisy_y[test_idx]

## Initialize Kernel

Create the Riemann Matern kernel. `nu` control the kernel smoothness like in the stadard Matern kernel. `neighbors` set the number of nearest neighbors in the KNN-based discrete approximation of the Laplace-Beltrami operator and construction of differentiable eigenfunctions. `modes` set the number of the eigenfunctions used to approximate the Riemann Matern kernel. As for classical *GPyTorch* kernels the Riemann Matern kernel can be decorated using with the *ScaleKernel* to have the signal variance parameter.

In [8]:
nu = 2
neighbors = 50
modes = 20
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(noisy_x, nu, neighbors, modes))

## Initialize Likelihood and GP Model

Define the type of likelihood and the GP model. The GP model takes as input the training set and and labels, the likelihood and the Riemann Matern Kernel.

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-8))
model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

## Model Hyperparameters Setting

In [10]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-6,
    'covar_module.base_kernel.epsilon': 0.1269,
    'covar_module.base_kernel.lengthscale': 0.3133,
    'covar_module.outputscale': 0.7133,
}
model.initialize(**hypers)

## Train Model

Train the GP model using the fast Lanczos precision matrix optimization. `lr` set the learning rate; `iters` defines the number of iteratios; `verbose` can be set to `True` for printing the parameters each iteration.

In [11]:
lr = 1e-2
iters = 100
verbose = True
loss = model.manifold_informed_train(lr, iters, verbose)

Iteration: 0, Loss: 111426.695, Noise Variance: 0.001, Signal Variance: 0.845, Lengthscale: 0.313, Epsilon: 0.127
Iteration: 1, Loss: 106983.477, Noise Variance: 0.001, Signal Variance: 0.848, Lengthscale: 0.316, Epsilon: 0.126
Iteration: 2, Loss: 102729.805, Noise Variance: 0.001, Signal Variance: 0.851, Lengthscale: 0.319, Epsilon: 0.125
Iteration: 3, Loss: 98660.617, Noise Variance: 0.001, Signal Variance: 0.854, Lengthscale: 0.321, Epsilon: 0.123
Iteration: 4, Loss: 94770.648, Noise Variance: 0.001, Signal Variance: 0.857, Lengthscale: 0.324, Epsilon: 0.122
Iteration: 5, Loss: 91054.477, Noise Variance: 0.001, Signal Variance: 0.860, Lengthscale: 0.327, Epsilon: 0.121
Iteration: 6, Loss: 87506.438, Noise Variance: 0.001, Signal Variance: 0.863, Lengthscale: 0.330, Epsilon: 0.120
Iteration: 7, Loss: 84120.867, Noise Variance: 0.001, Signal Variance: 0.866, Lengthscale: 0.332, Epsilon: 0.119
Iteration: 8, Loss: 80892.000, Noise Variance: 0.001, Signal Variance: 0.869, Lengthscale: 0.

Iteration: 72, Loss: 16726.186, Noise Variance: 0.001, Signal Variance: 0.995, Lengthscale: 0.469, Epsilon: 0.064
Iteration: 73, Loss: 16491.963, Noise Variance: 0.001, Signal Variance: 0.996, Lengthscale: 0.471, Epsilon: 0.064
Iteration: 74, Loss: 16263.781, Noise Variance: 0.001, Signal Variance: 0.997, Lengthscale: 0.472, Epsilon: 0.063
Iteration: 75, Loss: 16041.371, Noise Variance: 0.001, Signal Variance: 0.998, Lengthscale: 0.473, Epsilon: 0.062
Iteration: 76, Loss: 15824.535, Noise Variance: 0.001, Signal Variance: 0.999, Lengthscale: 0.475, Epsilon: 0.062
Iteration: 77, Loss: 15613.056, Noise Variance: 0.001, Signal Variance: 1.001, Lengthscale: 0.476, Epsilon: 0.061
Iteration: 78, Loss: 15406.742, Noise Variance: 0.001, Signal Variance: 1.002, Lengthscale: 0.478, Epsilon: 0.060
Iteration: 79, Loss: 15205.409, Noise Variance: 0.001, Signal Variance: 1.003, Lengthscale: 0.479, Epsilon: 0.060
Iteration: 80, Loss: 15008.869, Noise Variance: 0.001, Signal Variance: 1.004, Lengthsca

## Model Evaluation

In [12]:
%%capture
likelihood.eval()
model.eval()

Getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058). We compute the **mean**, the **standard deviation** and one **posterior sample**. In addition we evaluate the kernel at the first point of our dataset.

In [13]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = likelihood(model(sampled_x))
    mean = preds.mean
    std = preds.stddev
    posterior_sample = preds.sample()
    kernel_eval = kernel(sampled_x[0, :].unsqueeze(0), sampled_x).evaluate().squeeze()

## Plot Results

In [14]:
with torch.no_grad():
    # Bring data to cpu
    if use_cuda:
        sampled_x = sampled_x.cpu().numpy()
        sampled_y = sampled_y.cpu().numpy()
        train_x = train_x.cpu().numpy()
        test_x = test_x.cpu().numpy()
        kernel_eval = kernel_eval.cpu().numpy()
        posterior_sample = posterior_sample.cpu().numpy()
        mean = mean.cpu().numpy()
        std = std.cpu().numpy()

In [15]:
%%capture
%%bash
jupyter nbextension install --py mayavi --user

In [16]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere','scale_factor': 3e-3, 'color': (0, 0, 0)}

### Ground Truth with Sample Training Points

In [17]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=sampled_y)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)
# mlab.points3d(test_x[:,0], test_x[:,1], test_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Mean

In [18]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=mean)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Standard Deviation

In [19]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=std)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### One Posterior Sample

In [20]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=posterior_sample)
mlab.colorbar(orientation='vertical')
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Kernel Evaluation

In [21]:
mlab.figure(fgcolor=(0, 0, 0), bgcolor = (1,1,1))
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=kernel_eval)
mlab.colorbar(orientation='vertical')
mlab.points3d(sampled_x[0,0], sampled_x[0,1], sampled_x[0,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…